In [1]:
import pandas as pd
import gensim
import nltk
import gensim.corpora as corpora
from gensim.models import LdaModel, CoherenceModel, Phrases
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import re

In [2]:
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\owner\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\owner\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
data = pd.read_csv('dataset_expedia-hotels-com-reviews-scraper_2024.csv')

In [4]:
data.head()

,__typename,brandType,contentDirectFeedbackPromptId,customData/doWeEndorseIt,customData/hotel,hotelId,id,impressionAnalytics,impressionAnalytics/__typename,impressionAnalytics/event,...,translationInfo/loadingTranslationText,translationInfo/seeOriginalText,translationInfo/targetLocale,translationInfo/translatedBy/__typename,translationInfo/translatedBy/description,translationInfo/translationCallToActionLabel,travelers/0,travelers/1,travelers/2,travelers/3
0,PropertyReview,Expedia,NaN,no opinion,Prague Hotel Krystal,428588,66b63b958397934320ad7f75,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Traveled with family and small children,Traveled with family,NaN,NaN
1,PropertyReview,Expedia,NaN,no opinion,Prague Hotel Krystal,428588,66b503566a7cfe1353bdb91e,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Traveled with family,NaN,NaN,NaN
2,PropertyReview,Expedia,NaN,no opinion,Prague Hotel Krystal,428588,66b4d8d6e6073643fd1b9cdc,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Traveled with family and small children,NaN,NaN,NaN
3,PropertyReview,Expedia,NaN,no opinion,Prague Hotel Krystal,428588,66b3c0f33677b72a4489b05f,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Traveled with family and small children,NaN,NaN,NaN
4,PropertyReview,Expedia,NaN,no opinion,Prague Hotel Krystal,428588,66b36f292fa7b179c05b6637,NaN,ClientSideImpressionEventAnalytics,Scroll reviews,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Initialize stopwords and lemmatizer
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

# 1. Preprocessing Function (Simplified with Bigram within the function)
def preprocess(text):
    if not isinstance(text, str):
        text = ''
    
    # Remove special characters and numbers
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Convert to lowercase and tokenize
    tokens = simple_preprocess(text.lower(), deacc=True)
    
    # Lemmatize and remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

# First preprocess all texts
data['processed_text'] = data['text'].apply(preprocess)

In [30]:
# Then create and apply bigram model separately
texts = data['processed_text'].tolist()
bigram_phrases = Phrases(texts, min_count=2, threshold=10, delimiter='_')
bigram_model = Phrases.freeze(bigram_phrases)

# Apply bigram model to all texts
data['processed_text'] = [list(bigram_model[text]) for text in texts]

In [31]:
print(data['text'][0])

The property was too large for my liking. The food was okay some days and not great at all some days. Definitely the food needs a upgrade especially the Italian food. The pool area was beautiful, but the pools had mold. Pools need to be cleaned better. and the Beach lets talk about the beach. Its horrible it has tons of sea weed. If you are going to this hotel for the beach forget it. reroute yourself else were. I would give this hotel a 3 out of 5. Maybe next time I would prefer the Grand. Its newer.


In [32]:
print(data['processed_text'][0])

['property', 'large', 'liking', 'food', 'okay', 'day', 'great', 'day', 'definitely', 'food', 'need', 'upgrade', 'especially', 'italian', 'food', 'pool_area', 'beautiful', 'pool', 'mold', 'pool', 'need', 'cleaned', 'better', 'beach', 'let', 'talk', 'beach_horrible', 'ton', 'sea_weed', 'going', 'hotel', 'beach', 'forget', 'reroute', 'else', 'would_give', 'hotel', 'maybe', 'next_time', 'would_prefer', 'grand', 'newer']


In [33]:
# Create Dictionary and Corpus
dictionary = corpora.Dictionary(data['processed_text'])
# Filter out extremes
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in data['processed_text']]

In [41]:
# Build LDA Model
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=5, 
                     random_state=100,
                     update_every=1,
                     chunksize=500,
                     passes=200,
                     alpha='auto',
                     eta='auto',
                     per_word_topics=True)

In [43]:
# 4. Print the 25 most significant terms per topic
for i in range(5):
    print(f'Topic {i+1}:')
    print([term for term, _ in lda_model.show_topic(i, topn=25)])
    print('\n')

Topic 1:
['de', 'que', 'la', 'en', 'el', 'muy', 'e', 'excelente', 'para', 'hotel', 'no', 'los', 'un', 'con', 'se', 'de_la', 'todo', 'una', 'fue', 'al', 'por', 'el_servicio', 'servicio', 'del', 'la_comida']


Topic 2:
['good', 'beach', 'water', 'room', 'like', 'food', 'restaurant', 'seaweed', 'dirty', 'buffet', 'bad', 'also', 'ocean', 'pool', 'drink', 'nice', 'food_option', 'large', 'different', 'well', 'see', 'customer_service', 'clean', 'however', 'access']


Topic 3:
['resort', 'great', 'staff', 'property', 'service', 'food', 'pool', 'amazing', 'place', 'restaurant', 'nice', 'moon_palace', 'hotel', 'stay', 'clean', 'beautiful', 'kid', 'family', 'excellent', 'everything', 'friendly', 'experience', 'time', 'best', 'overall']


Topic 4:
['u', 'time', 'get', 'grand', 'hotel', 'would', 'way', 'stay', 'expedia', 'day', 'also', 'one', 'booked', 'make', 'pay', 'even', 'palace', 'sunrise', 'resort', 'membership', 'check', 'took', 'go', 'reservation', 'another']


Topic 5:
['room', 'get', 'hot

In [44]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics using pyLDAvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)

# Display the pyLDAvis visualization
pyLDAvis.display(vis)

In [45]:
# Get the topic distribution for each document
topic_distributions = []
for bow in corpus:
    topic_distribution = lda_model.get_document_topics(bow, minimum_probability=0)
    topic_distributions.append([weight for _, weight in topic_distribution])

# Create a DataFrame
topic_df = pd.DataFrame(topic_distributions, columns=[f'Topic_{i+1}' for i in range(lda_model.num_topics)])

# Add the processed text to the DataFrame
topic_df.insert(0, 'Processed_Text', data['processed_text'])

# Add a "Dominant_Topic" column with the name of the topic with the highest weight
topic_df['Dominant_Topic'] = topic_df.iloc[:, 1:].idxmax(axis=1)

topic_df


,Processed_Text,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Dominant_Topic
0,"[property, large, liking, food, okay, day, gre...",0.002303,0.699141,0.287054,0.006041,0.005460,Topic_2
1,"[stuff, friendly, food, excellent, variety_foo...",0.004323,0.462611,0.247208,0.275780,0.010079,Topic_2
2,"[big, organized, transportation, short, staffe...",0.010410,0.033683,0.902328,0.028339,0.025240,Topic_3
3,[good],0.035195,0.510687,0.284416,0.088313,0.081388,Topic_2
4,"[really, pushy, come, selling, resort, example...",0.006561,0.659430,0.300291,0.017746,0.015971,Topic_2
...,...,...,...,...,...,...,...
2407,"[el_complejo, de_lujo, la_atencion, del_person...",0.894127,0.021225,0.053017,0.016461,0.015171,Topic_1
2408,"[tres, bien, entretenu, personnel, tres, ecout...",0.592109,0.081530,0.203665,0.063230,0.059466,Topic_1
2409,"[se, hicieron, validos, los, usd, que, habian,...",0.892956,0.022080,0.053024,0.016680,0.015259,Topic_1
2410,[],0.058350,0.188777,0.471528,0.146413,0.134933,Topic_3


In [46]:
topic_df['Dominant_Topic'].value_counts()

Dominant_Topic
Topic_3    1731
Topic_1     206
Topic_2     170
Topic_5     156
Topic_4     149
Name: count, dtype: int64

In [47]:
# Function to print sample documents for a given topic
def print_sample_documents(topic_name, n_samples=5):
    # Filter the DataFrame based on the topic
    filtered_df = topic_df[topic_df['Dominant_Topic'] == topic_name]
    
    # Sample n_samples documents from the filtered DataFrame
    sample_documents = filtered_df['Processed_Text'].sample(n_samples, random_state=1)
    
    # Print the sampled documents
    for idx, text in enumerate(sample_documents, 1):
        print(f"Document {idx}: {text}\n")

In [54]:
print_sample_documents('Topic_1')

Document 1: ['servicio', 'personal', 'poco', 'amable', 'tardanza', 'excesiva', 'en', 'check', 'falta', 'de', 'limpieza', 'diaria', 'en', 'cuarto', 'cortes', 'de', 'luz', 'aire', 'acondicionado', 'malo', 'enfria', 'correctamente', 'la_comida', 'buena', 'disco', 'cerrada', 'unos', 'dia_en', 'bar', 'sky', 'musica', 'buena', 'los', 'cantantes', 'buenos']

Document 2: ['exelente', 'hotel', 'desde', 'que', 'llega', 'uno', 'te', 'atienden', 'de', 'lo_mejor', 'una', 'amabilidad', 'exelente']

Document 3: ['solisitud', 'del', 'renvolso', 'al', 'senor', 'oscar', 'eduardo', 'rengifo', 'de', 'palace_cancun', 'confimation', 'motivos', 'nunca', 'le', 'confirmaron', 'tiempo', 'tubo', 'que', 'reservar', 'en', 'otro', 'hotel', 'para', 'sarar', 'la_noche', 'por', 'favor', 'solisitamos', 'su', 'coperacion', 'gracias', 'el', 'nunca', 'llego', 'el_hotel', 'palace_cancun', 'confimation', 'el_dia']

Document 4: ['todo_el', 'personal_muy', 'amable', 'al', 'pendiente', 'de_los', 'huespedes', 'encanto', 'el_tra